In [1]:
replace_data = { "Spring": 0,
                 "Summer": 1,
                 "Autumn": 2,
                 "Winter": 3}

In [2]:
MAX_POWER = 100
MIN_POWER = 0

# MLP model

In [3]:
import tkinter as tk
import csv
import math
import tensorflow as tf
import pandas as pd

# Load your trained predictive model
model = tf.keras.models.load_model('/DIA/BOT/dnn.keras')

# Set up constants
SMALLEST_AMPLITUDE = 100  # Minimum amplitude for Winter
SPRING_AMPLITUDE = SMALLEST_AMPLITUDE * 1.5  # Proportional amplitude for Spring
SUMMER_AMPLITUDE = SMALLEST_AMPLITUDE * 2      # Proportional amplitude for Summer
AUTUMN_AMPLITUDE = SMALLEST_AMPLITUDE * 1.75  # Proportional amplitude for Autumn

# Initialize variables
current_amplitude = SMALLEST_AMPLITUDE
overall_day_count = 0

# Create the main window
root = tk.Tk()
# root.title("Sun Movement Simulation")

# Create a canvas
canvas = tk.Canvas(root, width=800, height=600, bg = 'green')
canvas.pack()

# Define the coordinates for the solar panels
panels = [
    (50, 400, 200, 550),   # Panel 1
    (220, 400, 370, 550),  # Panel 2
    (390, 400, 540, 550),  # Panel 3
    (560, 400, 710, 550),  # Panel 4
    (50, 200, 200, 350),   # Panel 5
    (560, 200, 710, 350)   # Panel 6
]



# Draw the solar panels
panel_objects = []
line_objects = []
for panel in panels:
    panel_obj = canvas.create_rectangle(panel, fill="navy", outline="white")
    panel_objects.append(panel_obj)
    panel_center_x = (panel[0] + panel[2]) / 2
    panel_center_y = (panel[1] + panel[3]) / 2
    line_obj = canvas.create_line(panel_center_x, panel_center_y, panel_center_x, panel_center_y, fill="red", width=2)
    line_objects.append(line_obj)

# Define the seasons
seasons = ["Spring", "Summer", "Autumn", "Winter"]

# Define the number of days per season
days_per_season = 1

# Draw the season markers
segment_width = canvas.winfo_width() // len(seasons)
for i, season in enumerate(seasons):
    canvas.create_text((i + 0.5) * segment_width, 20, text=season)

# Draw the sun
sun_radius = 50
sun_x = 0
sun_y = 100
sun = canvas.create_oval(sun_x - sun_radius, sun_y - sun_radius,
                          sun_x + sun_radius, sun_y + sun_radius,
                          fill="yellow", outline="orange")

# Open a CSV file to store data
data_file = open("sun_movement_data.csv", "a", newline="")
csv_writer = csv.writer(data_file)

# Add a header row only if the file is newly created
if data_file.tell() == 0:
    csv_writer.writerow(["day", "time", "season", "sun_x", "sun_y", "predicted_sun_x", "predicted_sun_y"])

# Define the angles for the sun during different seasons
sun_angles = {"Spring": 0, "Summer": 90, "Autumn": 180, "Winter": 270}
current_angle = 0

# Define the animation function with adjusted sun position calculations
def animate_sun(day, season_index, season, time=0):
    global sun_x, sun_y, current_angle, current_amplitude, overall_day_count
    canvas.move(sun, 2, 0)  # Move the sun horizontally
    sun_x += 2  # Update the x-coordinate of the sun

    # Calculate the sun's y-coordinate using a sine function for an arc-like motion
    radians = math.radians(current_angle)
    distance_from_center = 300 + current_amplitude * math.cos(radians)
    sun_y = distance_from_center * math.sin(math.radians(sun_x / 800) * 360)

    # Change the angle and amplitude when moving from one season to another
    if day > 0 and day % (days_per_season) == 0:
        current_angle = sun_angles[season]
        current_amplitude = {
            'Spring': SPRING_AMPLITUDE,
            'Summer': SUMMER_AMPLITUDE,
            'Autumn': AUTUMN_AMPLITUDE,
            'Winter': SMALLEST_AMPLITUDE
        }[season]
        print(f"New Season: {season}, Angle: {current_angle}, Amplitude: {current_amplitude}")

    # Check if the day has ended
    if sun_x >= canvas.winfo_width() - sun_radius:
        sun_x = -sun_radius  # Reset the x-coordinate to the left edge
        sun_y = 100  # Reset the y-coordinate to its initial value
        day += 1  # Increment the day
        time = 0  # Reset time for a new day
        overall_day_count += 1

        # Reset overall_day_count to 1 after 360 days
        if overall_day_count > 360:
            overall_day_count = 1

        # Check if a new season starts
        if day % days_per_season == 0:
            season_index = (season_index + 1) % len(seasons)  # Cycle through seasons
            season = seasons[season_index]
            print(f"Season: {season}")

    canvas.coords(sun, sun_x - sun_radius, sun_y - sun_radius,
                  sun_x + sun_radius, sun_y + sun_radius)  # Update the sun's position

    # Use the predictive model to update the positions of the solar panels' red lines
    predicted_sun_x, predicted_sun_y = predict_sun_position(overall_day_count, time, season)
    for line_obj, panel in zip(line_objects, panels):
        panel_center_x = (panel[0] + panel[2]) / 2
        panel_center_y = (panel[1] + panel[3]) / 2

        # Calculate the distance between the line endpoint and the sun
        distance = math.sqrt((predicted_sun_x - sun_x) ** 2 + (predicted_sun_y - sun_y) ** 2)

        # Calculate the power level based on the distance
        power_level = MAX_POWER - (distance / 8)  # Adjust the denominator as needed

        # Ensure power level is within the defined range
        power_level = max(MIN_POWER, min(MAX_POWER, power_level))

        # Set the brightness of the line based on the power level
        brightness = int(power_level * 255 / MAX_POWER)
        line_color = "#%02x%02x%02x" % (255, brightness, brightness)

        # Update the line's color
        canvas.itemconfig(line_obj, fill=line_color)

        # Update the panel's color based on the power level
        panel_color = "#%02x%02x%02x" % (255, 255 - brightness, 255 - brightness)
        canvas.itemconfig(panel_objects[line_objects.index(line_obj)], fill=panel_color)

        canvas.coords(line_obj, panel_center_x, panel_center_y, predicted_sun_x, predicted_sun_y)

    # Save data to CSV file
    csv_writer.writerow([overall_day_count, time, season, sun_x, sun_y, predicted_sun_x, predicted_sun_y])

    time += 1  # Increment time within the day
    root.after(75, animate_sun, day, season_index, season, time)  # Schedule the next frame
# Function to predict the sun's position using the model
def predict_sun_position(day, time, season):
    # Create a DataFrame with the input data
    input_data = pd.DataFrame({'day': [day], 'time': [time], 'season': [season]})
    input_data = input_data.replace({"season": replace_data})
    # Use the model to predict the sun's position
    predicted_sun_position = model.predict(input_data)
    # Extract the predicted sun position
    predicted_sun_x, predicted_sun_y = predicted_sun_position[0]
    return predicted_sun_x, predicted_sun_y

# Start the animation
animate_sun(0, 0, seasons[0])

# Run the main event loop
root.mainloop()

# Close the CSV file
data_file.close()


E:\DIA\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})


Season: Summer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
New Season: Summer, Angle: 90, Amplitude: 200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
New Season: Summer, Angle: 90, Amplitude: 200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
New Season: Summer, Angle: 90, Amplitude: 200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
New Season: Summer, Angle: 90, Amplitude: 200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})


...

C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})


New Season: Summer, Angle: 90, Amplitude: 200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
New Season: Summer, Angle: 90, Amplitude: 200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step

C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
New Season: Summer, Angle: 90, Amplitude: 200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


C:\Users\crdka\AppData\Local\Temp\ipykernel_10580\2582246234.py:162: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})


# Random Forest

In [1]:
import tkinter as tk
import csv
import math
import tensorflow as tf
import pandas as pd
import pickle

# Load your trained predictive model
with open('random_forest_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Set up constants
SMALLEST_AMPLITUDE = 100  # Minimum amplitude for Winter
SPRING_AMPLITUDE = SMALLEST_AMPLITUDE * 1.5  # Proportional amplitude for Spring
SUMMER_AMPLITUDE = SMALLEST_AMPLITUDE * 2      # Proportional amplitude for Summer
AUTUMN_AMPLITUDE = SMALLEST_AMPLITUDE * 1.75  # Proportional amplitude for Autumn

# Initialize variables
current_amplitude = SMALLEST_AMPLITUDE
overall_day_count = 0

# Create the main window
root = tk.Tk()
# root.title("Sun Movement Simulation")

# Create a canvas
canvas = tk.Canvas(root, width=800, height=600, bg = 'green')
canvas.pack()

# Define the coordinates for the solar panels
panels = [
    (50, 400, 200, 550),   # Panel 1
    (220, 400, 370, 550),  # Panel 2
    (390, 400, 540, 550),  # Panel 3
    (560, 400, 710, 550),  # Panel 4
    (50, 200, 200, 350),   # Panel 5
    (560, 200, 710, 350)   # Panel 6
]



# Draw the solar panels
panel_objects = []
line_objects = []
for panel in panels:
    panel_obj = canvas.create_rectangle(panel, fill="navy", outline="white")
    panel_objects.append(panel_obj)
    panel_center_x = (panel[0] + panel[2]) / 2
    panel_center_y = (panel[1] + panel[3]) / 2
    line_obj = canvas.create_line(panel_center_x, panel_center_y, panel_center_x, panel_center_y, fill="red", width=2)
    line_objects.append(line_obj)

# Define the seasons
seasons = ["Spring", "Summer", "Autumn", "Winter"]

# Define the number of days per season
days_per_season = 1

# Draw the season markers
segment_width = canvas.winfo_width() // len(seasons)
for i, season in enumerate(seasons):
    canvas.create_text((i + 0.5) * segment_width, 20, text=season)

# Draw the sun
sun_radius = 50
sun_x = 0
sun_y = 100
sun = canvas.create_oval(sun_x - sun_radius, sun_y - sun_radius,
                          sun_x + sun_radius, sun_y + sun_radius,
                          fill="yellow", outline="orange")

# Open a CSV file to store data
data_file = open("sun_movement_data.csv", "a", newline="")
csv_writer = csv.writer(data_file)

# Add a header row only if the file is newly created
if data_file.tell() == 0:
    csv_writer.writerow(["day", "time", "season", "sun_x", "sun_y", "predicted_sun_x", "predicted_sun_y"])

# Define the angles for the sun during different seasons
sun_angles = {"Spring": 0, "Summer": 90, "Autumn": 180, "Winter": 270}
current_angle = 0

# Define the animation function with adjusted sun position calculations
def animate_sun(day, season_index, season, time=0):
    global sun_x, sun_y, current_angle, current_amplitude, overall_day_count
    canvas.move(sun, 2, 0)  # Move the sun horizontally
    sun_x += 2  # Update the x-coordinate of the sun

    # Calculate the sun's y-coordinate using a sine function for an arc-like motion
    radians = math.radians(current_angle)
    distance_from_center = 300 + current_amplitude * math.cos(radians)
    sun_y = distance_from_center * math.sin(math.radians(sun_x / 800) * 360)

    # Change the angle and amplitude when moving from one season to another
    if day > 0 and day % (days_per_season) == 0:
        current_angle = sun_angles[season]
        current_amplitude = {
            'Spring': SPRING_AMPLITUDE,
            'Summer': SUMMER_AMPLITUDE,
            'Autumn': AUTUMN_AMPLITUDE,
            'Winter': SMALLEST_AMPLITUDE
        }[season]
        print(f"New Season: {season}, Angle: {current_angle}, Amplitude: {current_amplitude}")

    # Check if the day has ended
    if sun_x >= canvas.winfo_width() - sun_radius:
        sun_x = -sun_radius  # Reset the x-coordinate to the left edge
        sun_y = 100  # Reset the y-coordinate to its initial value
        day += 1  # Increment the day
        time = 0  # Reset time for a new day
        overall_day_count += 1

        # Reset overall_day_count to 1 after 360 days
        if overall_day_count > 360:
            overall_day_count = 1

        # Check if a new season starts
        if day % days_per_season == 0:
            season_index = (season_index + 1) % len(seasons)  # Cycle through seasons
            season = seasons[season_index]
            print(f"Season: {season}")

    canvas.coords(sun, sun_x - sun_radius, sun_y - sun_radius,
                  sun_x + sun_radius, sun_y + sun_radius)  # Update the sun's position

    # Use the predictive model to update the positions of the solar panels' red lines
    predicted_sun_x, predicted_sun_y = predict_sun_position(overall_day_count, time, season)
    for line_obj, panel in zip(line_objects, panels):
        panel_center_x = (panel[0] + panel[2]) / 2
        panel_center_y = (panel[1] + panel[3]) / 2

        # Calculate the distance between the line endpoint and the sun
        distance = math.sqrt((predicted_sun_x - sun_x) ** 2 + (predicted_sun_y - sun_y) ** 2)

        # Calculate the power level based on the distance
        power_level = MAX_POWER - (distance / 8)  # Adjust the denominator as needed

        # Ensure power level is within the defined range
        power_level = max(MIN_POWER, min(MAX_POWER, power_level))

        # Set the brightness of the line based on the power level
        brightness = int(power_level * 255 / MAX_POWER)
        line_color = "#%02x%02x%02x" % (255, brightness, brightness)

        # Update the line's color
        canvas.itemconfig(line_obj, fill=line_color)

        # Update the panel's color based on the power level
        panel_color = "#%02x%02x%02x" % (255, 255 - brightness, 255 - brightness)
        canvas.itemconfig(panel_objects[line_objects.index(line_obj)], fill=panel_color)

        canvas.coords(line_obj, panel_center_x, panel_center_y, predicted_sun_x, predicted_sun_y)

    # Save data to CSV file
    csv_writer.writerow([overall_day_count, time, season, sun_x, sun_y, predicted_sun_x, predicted_sun_y])

    time += 1  # Increment time within the day
    root.after(75, animate_sun, day, season_index, season, time)  # Schedule the next frame
# Function to predict the sun's position using the model
def predict_sun_position(day, time, season):
    # Create a DataFrame with the input data
    input_data = pd.DataFrame({'day': [day], 'time': [time], 'season': [season]})
    input_data = input_data.replace({"season": replace_data})
    # Use the model to predict the sun's position
    predicted_sun_position = loaded_model.predict(input_data)
    # Extract the predicted sun position
    predicted_sun_x, predicted_sun_y = predicted_sun_position[0]
    return predicted_sun_x, predicted_sun_y

# Start the animation
animate_sun(0, 0, seasons[0])

# Run the main event loop
root.mainloop()

# Close the CSV file
data_file.close()


Season: Summer


NameError: name 'replace_data' is not defined

# XGBoost

In [18]:
import tkinter as tk
import csv
import math
import tensorflow as tf
import pandas as pd
import pickle
import xgboost

# Load your trained predictive model
with open('xgboost_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Set up constants
SMALLEST_AMPLITUDE = 100  # Minimum amplitude for Winter
SPRING_AMPLITUDE = SMALLEST_AMPLITUDE * 1.5  # Proportional amplitude for Spring
SUMMER_AMPLITUDE = SMALLEST_AMPLITUDE * 2      # Proportional amplitude for Summer
AUTUMN_AMPLITUDE = SMALLEST_AMPLITUDE * 1.75  # Proportional amplitude for Autumn

# Initialize variables
current_amplitude = SMALLEST_AMPLITUDE
overall_day_count = 0

# Create the main window
root = tk.Tk()
# root.title("Sun Movement Simulation")

# Create a canvas
canvas = tk.Canvas(root, width=800, height=600, bg = 'green')
canvas.pack()

# Define the coordinates for the solar panels
panels = [
    (50, 400, 200, 550),   # Panel 1
    (220, 400, 370, 550),  # Panel 2
    (390, 400, 540, 550),  # Panel 3
    (560, 400, 710, 550),  # Panel 4
    (50, 200, 200, 350),   # Panel 5
    (560, 200, 710, 350)   # Panel 6
]



# Draw the solar panels
panel_objects = []
line_objects = []
for panel in panels:
    panel_obj = canvas.create_rectangle(panel, fill="navy", outline="white")
    panel_objects.append(panel_obj)
    panel_center_x = (panel[0] + panel[2]) / 2
    panel_center_y = (panel[1] + panel[3]) / 2
    line_obj = canvas.create_line(panel_center_x, panel_center_y, panel_center_x, panel_center_y, fill="red", width=2)
    line_objects.append(line_obj)

# Define the seasons
seasons = ["Spring", "Summer", "Autumn", "Winter"]

# Define the number of days per season
days_per_season = 1

# Draw the season markers
segment_width = canvas.winfo_width() // len(seasons)
for i, season in enumerate(seasons):
    canvas.create_text((i + 0.5) * segment_width, 20, text=season)

# Draw the sun
sun_radius = 50
sun_x = 0
sun_y = 100
sun = canvas.create_oval(sun_x - sun_radius, sun_y - sun_radius,
                          sun_x + sun_radius, sun_y + sun_radius,
                          fill="yellow", outline="orange")

# Open a CSV file to store data
data_file = open("sun_movement_data.csv", "a", newline="")
csv_writer = csv.writer(data_file)

# Add a header row only if the file is newly created
if data_file.tell() == 0:
    csv_writer.writerow(["day", "time", "season", "sun_x", "sun_y", "predicted_sun_x", "predicted_sun_y"])

# Define the angles for the sun during different seasons
sun_angles = {"Spring": 0, "Summer": 90, "Autumn": 180, "Winter": 270}
current_angle = 0

# Define the animation function with adjusted sun position calculations
def animate_sun(day, season_index, season, time=0):
    global sun_x, sun_y, current_angle, current_amplitude, overall_day_count
    canvas.move(sun, 2, 0)  # Move the sun horizontally
    sun_x += 2  # Update the x-coordinate of the sun

    # Calculate the sun's y-coordinate using a sine function for an arc-like motion
    radians = math.radians(current_angle)
    distance_from_center = 300 + current_amplitude * math.cos(radians)
    sun_y = distance_from_center * math.sin(math.radians(sun_x / 800) * 360)

    # Change the angle and amplitude when moving from one season to another
    if day > 0 and day % (days_per_season) == 0:
        current_angle = sun_angles[season]
        current_amplitude = {
            'Spring': SPRING_AMPLITUDE,
            'Summer': SUMMER_AMPLITUDE,
            'Autumn': AUTUMN_AMPLITUDE,
            'Winter': SMALLEST_AMPLITUDE
        }[season]
        print(f"New Season: {season}, Angle: {current_angle}, Amplitude: {current_amplitude}")

    # Check if the day has ended
    if sun_x >= canvas.winfo_width() - sun_radius:
        sun_x = -sun_radius  # Reset the x-coordinate to the left edge
        sun_y = 100  # Reset the y-coordinate to its initial value
        day += 1  # Increment the day
        time = 0  # Reset time for a new day
        overall_day_count += 1

        # Reset overall_day_count to 1 after 360 days
        if overall_day_count > 360:
            overall_day_count = 1

        # Check if a new season starts
        if day % days_per_season == 0:
            season_index = (season_index + 1) % len(seasons)  # Cycle through seasons
            season = seasons[season_index]
            print(f"Season: {season}")

    canvas.coords(sun, sun_x - sun_radius, sun_y - sun_radius,
                  sun_x + sun_radius, sun_y + sun_radius)  # Update the sun's position

    # Use the predictive model to update the positions of the solar panels' red lines
    predicted_sun_x, predicted_sun_y = predict_sun_position(overall_day_count, time, season)
    for line_obj, panel in zip(line_objects, panels):
        panel_center_x = (panel[0] + panel[2]) / 2
        panel_center_y = (panel[1] + panel[3]) / 2

        # Calculate the distance between the line endpoint and the sun
        distance = math.sqrt((predicted_sun_x - sun_x) ** 2 + (predicted_sun_y - sun_y) ** 2)

        # Calculate the power level based on the distance
        power_level = MAX_POWER - (distance / 8)  # Adjust the denominator as needed

        # Ensure power level is within the defined range
        power_level = max(MIN_POWER, min(MAX_POWER, power_level))

        # Set the brightness of the line based on the power level
        brightness = int(power_level * 255 / MAX_POWER)
        line_color = "#%02x%02x%02x" % (255, brightness, brightness)

        # Update the line's color
        canvas.itemconfig(line_obj, fill=line_color)

        # Update the panel's color based on the power level
        panel_color = "#%02x%02x%02x" % (255, 255 - brightness, 255 - brightness)
        canvas.itemconfig(panel_objects[line_objects.index(line_obj)], fill=panel_color)

        canvas.coords(line_obj, panel_center_x, panel_center_y, predicted_sun_x, predicted_sun_y)

    # Save data to CSV file
    csv_writer.writerow([overall_day_count, time, season, sun_x, sun_y, predicted_sun_x, predicted_sun_y])

    time += 1  # Increment time within the day
    root.after(75, animate_sun, day, season_index, season, time)  # Schedule the next frame
# Function to predict the sun's position using the model
def predict_sun_position(day, time, season):
    # Create a DataFrame with the input data
    input_data = pd.DataFrame({'day': [day], 'time': [time], 'season': [season]})
    input_data = input_data.replace({"season": replace_data})
    # Use the model to predict the sun's position
    predicted_sun_position = loaded_model.predict(input_data)
    # Extract the predicted sun position
    predicted_sun_x, predicted_sun_y = predicted_sun_position[0]
    return predicted_sun_x, predicted_sun_y

# Start the animation
animate_sun(0, 0, seasons[0])

# Run the main event loop
root.mainloop()

# Close the CSV file
data_file.close()


Season: Summer
New Season: Summer, Angle: 90, Amplitude: 200
New Season: Summer, Angle: 90, Amplitude: 200


C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

New Season: Summer, Angle: 90, Amplitude: 200
New Season: Summer, Angle: 90, Amplitude: 200
New Season: Summer, Angle: 90, Amplitude: 200


C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

New Season: Summer, Angle: 90, Amplitude: 200
New Season: Summer, Angle: 90, Amplitude: 200
New Season: Summer, Angle: 90, Amplitude: 200


...

C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

New Season: Winter, Angle: 270, Amplitude: 100
New Season: Winter, Angle: 270, Amplitude: 100
New Season: Winter, Angle: 270, Amplitude: 100


C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

New Season: Winter, Angle: 270, Amplitude: 100
New Season: Winter, Angle: 270, Amplitude: 100
New Season: Winter, Angle: 270, Amplitude: 100


C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\1839991698.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

# LGBM Model

In [19]:
import tkinter as tk
import csv
import math
import tensorflow as tf
import pandas as pd
import pickle
import xgboost

# Load your trained predictive model
with open('lgbm_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Set up constants
SMALLEST_AMPLITUDE = 100  # Minimum amplitude for Winter
SPRING_AMPLITUDE = SMALLEST_AMPLITUDE * 1.5  # Proportional amplitude for Spring
SUMMER_AMPLITUDE = SMALLEST_AMPLITUDE * 2      # Proportional amplitude for Summer
AUTUMN_AMPLITUDE = SMALLEST_AMPLITUDE * 1.75  # Proportional amplitude for Autumn

# Initialize variables
current_amplitude = SMALLEST_AMPLITUDE
overall_day_count = 0

# Create the main window
root = tk.Tk()
# root.title("Sun Movement Simulation")

# Create a canvas
canvas = tk.Canvas(root, width=800, height=600, bg = 'green')
canvas.pack()

# Define the coordinates for the solar panels
panels = [
    (50, 400, 200, 550),   # Panel 1
    (220, 400, 370, 550),  # Panel 2
    (390, 400, 540, 550),  # Panel 3
    (560, 400, 710, 550),  # Panel 4
    (50, 200, 200, 350),   # Panel 5
    (560, 200, 710, 350)   # Panel 6
]



# Draw the solar panels
panel_objects = []
line_objects = []
for panel in panels:
    panel_obj = canvas.create_rectangle(panel, fill="navy", outline="white")
    panel_objects.append(panel_obj)
    panel_center_x = (panel[0] + panel[2]) / 2
    panel_center_y = (panel[1] + panel[3]) / 2
    line_obj = canvas.create_line(panel_center_x, panel_center_y, panel_center_x, panel_center_y, fill="red", width=2)
    line_objects.append(line_obj)

# Define the seasons
seasons = ["Spring", "Summer", "Autumn", "Winter"]

# Define the number of days per season
days_per_season = 1

# Draw the season markers
segment_width = canvas.winfo_width() // len(seasons)
for i, season in enumerate(seasons):
    canvas.create_text((i + 0.5) * segment_width, 20, text=season)

# Draw the sun
sun_radius = 50
sun_x = 0
sun_y = 100
sun = canvas.create_oval(sun_x - sun_radius, sun_y - sun_radius,
                          sun_x + sun_radius, sun_y + sun_radius,
                          fill="yellow", outline="orange")

# Open a CSV file to store data
data_file = open("sun_movement_data.csv", "a", newline="")
csv_writer = csv.writer(data_file)

# Add a header row only if the file is newly created
if data_file.tell() == 0:
    csv_writer.writerow(["day", "time", "season", "sun_x", "sun_y", "predicted_sun_x", "predicted_sun_y"])

# Define the angles for the sun during different seasons
sun_angles = {"Spring": 0, "Summer": 90, "Autumn": 180, "Winter": 270}
current_angle = 0

# Define the animation function with adjusted sun position calculations
def animate_sun(day, season_index, season, time=0):
    global sun_x, sun_y, current_angle, current_amplitude, overall_day_count
    canvas.move(sun, 2, 0)  # Move the sun horizontally
    sun_x += 2  # Update the x-coordinate of the sun

    # Calculate the sun's y-coordinate using a sine function for an arc-like motion
    radians = math.radians(current_angle)
    distance_from_center = 300 + current_amplitude * math.cos(radians)
    sun_y = distance_from_center * math.sin(math.radians(sun_x / 800) * 360)

    # Change the angle and amplitude when moving from one season to another
    if day > 0 and day % (days_per_season) == 0:
        current_angle = sun_angles[season]
        current_amplitude = {
            'Spring': SPRING_AMPLITUDE,
            'Summer': SUMMER_AMPLITUDE,
            'Autumn': AUTUMN_AMPLITUDE,
            'Winter': SMALLEST_AMPLITUDE
        }[season]
        print(f"New Season: {season}, Angle: {current_angle}, Amplitude: {current_amplitude}")

    # Check if the day has ended
    if sun_x >= canvas.winfo_width() - sun_radius:
        sun_x = -sun_radius  # Reset the x-coordinate to the left edge
        sun_y = 100  # Reset the y-coordinate to its initial value
        day += 1  # Increment the day
        time = 0  # Reset time for a new day
        overall_day_count += 1

        # Reset overall_day_count to 1 after 360 days
        if overall_day_count > 360:
            overall_day_count = 1

        # Check if a new season starts
        if day % days_per_season == 0:
            season_index = (season_index + 1) % len(seasons)  # Cycle through seasons
            season = seasons[season_index]
            print(f"Season: {season}")

    canvas.coords(sun, sun_x - sun_radius, sun_y - sun_radius,
                  sun_x + sun_radius, sun_y + sun_radius)  # Update the sun's position

    # Use the predictive model to update the positions of the solar panels' red lines
    predicted_sun_x, predicted_sun_y = predict_sun_position(overall_day_count, time, season)
    for line_obj, panel in zip(line_objects, panels):
        panel_center_x = (panel[0] + panel[2]) / 2
        panel_center_y = (panel[1] + panel[3]) / 2

        # Calculate the distance between the line endpoint and the sun
        distance = math.sqrt((predicted_sun_x - sun_x) ** 2 + (predicted_sun_y - sun_y) ** 2)

        # Calculate the power level based on the distance
        power_level = MAX_POWER - (distance / 8)  # Adjust the denominator as needed

        # Ensure power level is within the defined range
        power_level = max(MIN_POWER, min(MAX_POWER, power_level))

        # Set the brightness of the line based on the power level
        brightness = int(power_level * 255 / MAX_POWER)
        line_color = "#%02x%02x%02x" % (255, brightness, brightness)

        # Update the line's color
        canvas.itemconfig(line_obj, fill=line_color)

        # Update the panel's color based on the power level
        panel_color = "#%02x%02x%02x" % (255, 255 - brightness, 255 - brightness)
        canvas.itemconfig(panel_objects[line_objects.index(line_obj)], fill=panel_color)

        canvas.coords(line_obj, panel_center_x, panel_center_y, predicted_sun_x, predicted_sun_y)

    # Save data to CSV file
    csv_writer.writerow([overall_day_count, time, season, sun_x, sun_y, predicted_sun_x, predicted_sun_y])

    time += 1  # Increment time within the day
    root.after(75, animate_sun, day, season_index, season, time)  # Schedule the next frame
# Function to predict the sun's position using the model
def predict_sun_position(day, time, season):
    # Create a DataFrame with the input data
    input_data = pd.DataFrame({'day': [day], 'time': [time], 'season': [season]})
    input_data = input_data.replace({"season": replace_data})
    # Use the model to predict the sun's position
    predicted_sun_position = loaded_model.predict(input_data)
    # Extract the predicted sun position
    predicted_sun_x, predicted_sun_y = predicted_sun_position[0]
    return predicted_sun_x, predicted_sun_y

# Start the animation
animate_sun(0, 0, seasons[0])

# Run the main event loop
root.mainloop()

# Close the CSV file
data_file.close()


Season: Summer
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Summer, Angle: 90, Amplitude: 200
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Summer, Angle: 90, Amplitude: 200
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

New Season: Summer, Angle: 90, Amplitude: 200
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Summer, Angle: 90, Amplitude: 200
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Summer, Angle: 90, Amplitude: 200
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

New Season: Summer, Angle: 90, Amplitude: 200
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Summer, Angle: 90, Amplitude: 200
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Summer, Angle: 90, Amplitude: 200
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


...

C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

New Season: Winter, Angle: 270, Amplitude: 100
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Winter, Angle: 270, Amplitude: 100
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Winter, Angle: 270, Amplitude: 100
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

New Season: Winter, Angle: 270, Amplitude: 100
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Winter, Angle: 270, Amplitude: 100
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
New Season: Winter, Angle: 270, Amplitude: 100
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  input_data = input_data.replace({"season": replace_data})
C:\Users\crdka\AppData\Local\Temp\ipykernel_13584\3355317641.py:165: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old